In [ ]:
from resources.workspace import *
from IPython.display import display
from scipy.integrate import odeint
import copy

%matplotlib inline

# Chaos and the dynamical properties of ensemble-based covariances

Chaos is commonly understood by the <b>butterfly effect</b>; "A buttefly that flaps its wings in Brazil can "cause" a hurricane in Texas". As opposed to the opinions of Descartes/Newton/Laplace, chaos effectively means that even in a deterministic (non-stochastic) universe, we can only predict "so far" into the future.  We will introduce two very typical "toy" models that exhibit these features.

## Examples of models chaotic systems

### The "Lorenz-95" model

The [Lorenz 95/ 96 system](http://eaps4.mit.edu/research/Lorenz/Predicability_a_Problem_2006.pdf) is a one dimensional model, designed to simulate atmospheric convection.  Each variable <span style='font-size:1.25em'>$x^j$ </span> can be considered some atmospheric quantity in one of $m$ sectors along a single lattitude.  The differential equation for <span style='font-size:1.25em'>$x^j$ </span> reads,
<h3>$$
\frac{{\rm d} x^j}{{\rm d} t} \triangleq -x^{j-1} x^{j-2} + x^{j-1}x^{j+1} - x^j + F,
$$</h3>
where all indices $j$ are taken modulo $m$. 

There are **no accurate physics** represented in this model.  Rather, the model only seeks to capture qualitative features of the atmosphere, in that:
<ul>
    <li> there is external forcing, determined by a parameter $F$;</li>
    <li> there is internal dissipation, simulated by linear terms;</li>
    <li> there is advection, simulated by quadratic terms.</li>
</ul>

The number of sectors $m$ is assumed to be at least $m=4$ but typically it is taken that $m=40$. See the following link for
[further description](resources/DA_intro.pdf#page=23).

** Exc 4.2**: The system above has an easy to find fixed point, i.e., a point <span style='font-size:1.25em'>$\mathbf{x}_0$</span> such that
<h3>$$ \frac{{\rm d}}{{\rm d} t} \mathbf{x}_0 \equiv 0$$ </h3>

Can you identify one?

In [ ]:
# Example solution

# show_answer('fixed_point')

A fixed point is **stable** if for any perturbations sufficiently small, a trajectory evolved from this perturbation must be attracted to the fixed point.  That means, all nearby solutions will settle to a solution that doesn't change in time.  A classification of various fixed point dynamics is illustrated in the figure below.

<div style='width:900px'>
<img src="./resources/Stability_Diagram.png">
</div>

**By Freesodas (Gimp) [<a href="http://www.gnu.org/copyleft/fdl.html">GFDL</a> or <a href="https://creativecommons.org/licenses/by-sa/4.0">CC BY-SA 4.0</a>], <a href="https://commons.wikimedia.org/wiki/File:Stability_Diagram.png">via Wikimedia Commons</a>**

In the Lorenz-95 model, different values for $F$ will produce different behaviors in the model.  For some values of $F$, the fixed point from **Exc 4.2** is stable.  For some values of $F$, perturbations won't be drawn to the fixed point, but will settle to another kind of **steady behavior**.  For some values of $F$, small perturbations will behave wildy, with growth and decay that is difficult to predict.

**Exc 4.4**: Run the code below to interactively plot the behavior of the Lorenz-95 sytem.  The figure on the left hand side below plots a time lapse of the values of <span style='font-size:1.25em'>$x^j$</span> on the $y$-axis, while the $x$-axis varies each sector $j$, modulo $m=10$.  The time variable is given by "T" below. 

The figure on the right hand side below plots the time series of the total engergy of the system, defined by
<h3>$$\begin{align}
\mathbf{E}(\mathbf{x}) &= \frac{1}{2} \sum_{j=1}^m \left(\mathbf{x}^j\right)^2.
\end{align}$$
</h3>
Note that for $T>30$, we only plot the time series in the interval $[T-30, T]$.

For each value of $F$, we initialize the model with a small perturbation of size "eps=0.5" to the fixed point found in **Exc 4.2**.  Answer the following questions:
<ol>
   <li> For what values of $F$ does it look like the fixed point is stable?  How is this reflected in the energy in the system?</li>
   <li> For what values of $F$ does it look like the system settles to periodic motion? How is this reflected in the energy in the system?</li>
   <li> For what values of $F$ does the evolution become "chaotic"? How is this reflected in the energy in the system?
   <li> The classical choice for $F$ is $F=8$.  What kind of behavior is exhibited?
<ol>

In [ ]:
# For all i, any n: s(x,n) := x[i+n], circularly.
def s(x,n):
    return np.roll(x,-n)

def animate_lorenz_95(F=0.8,T=0):
    # Initial conditions: perturbations
    eps=.5
    m=10
    x0 = ones(m)
    x0 = x0 * F
    x0[0] += eps
    
    def dxdt(x,t):
        return (s(x,1)-s(x,-2))*s(x,-1) - x + F
    
    tt = linspace(0, T, int(T/.1) + 1)
    xx = odeint(lambda x,t: dxdt(x,t), x0, tt)
    energy =  .5 * np.sum(xx*2, axis=1)
    xx = np.concatenate([xx,
                         np.reshape(xx[:,0], [len(xx[:,0]), 1])],axis=1)
    

    # Plot multiple
    fig = plt.figure(figsize=(16,6))
    ax1 = fig.add_axes([.08, .095,  .4, .89])
    ax2 = fig.add_axes([.525, .095, .4, .89])

    Lag = 4
    colors = plt.cm.cubehelix(0.1+0.6*linspace(0,1,Lag))
    for k in range(Lag,0,-1):
        ax1.plot(xx[max(0,len(xx)-k)],c=colors[Lag-k])

    ax1.set_ylim(-10,20)
    ax1.set_xlabel(r'Sector $j$', size=30)
    ax1.set_xticks(range(0,12,2))
    ax1.set_ylabel(r'$x^j$', size=30)
    ax1.tick_params(
        labelsize=20)
    ax2.plot(tt[-300:], energy[-300:])
    ax2.set_xlabel('Time T',size=30)
    ax2.yaxis.set_label_position("right")
    ax2.set_ylabel('Total Energy',size=30, rotation=270)
    ax2.yaxis.set_label_coords(1.175,.5)
    ax2.tick_params(
        axis='x',
        labelsize=20)
    ax2.tick_params(
        axis='y',
        labelsize=20,
        right=True,
        labelright=True,
        left=False,
        labelleft=False
    )
    tics = [np.round(i,decimals=1) for i in linspace(np.min(energy[-300:]) - 1, np.max(energy[-300:]) + 1, 5)]
    ax2.set_yticks(tics)
    ax2.set_yticklabels([str(i).zfill(2) for i in tics])
    
    plt.show()
    
interact(animate_lorenz_95,T=(0.2,60.2,0.1),F=(0,12,.2));

### The Lorenz (1963) system

The <b>[Lorenz-63 system](https://journals.ametsoc.org/doi/abs/10.1175/1520-0469%281963%29020%3C0130%3ADNF%3E2.0.CO%3B2)</b>, commonly known as the "butterfly attractor", is a simplified mathematical model for atmospheric convection respresenting real physics.  The Lorenz equations are derived from the Oberbeck-Boussinesq approximation to the equations describing fluid circulation in a shallow layer of fluid, heated uniformly from below and cooled uniformly from above - this describes Rayleigh-Bénard convection.

The Lorenz-63 system is given by the 3 coupled ordinary differential equations (ODE):
<h3>
$$
\begin{aligned}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{aligned}
$$
</h3>
where 
<h3>$$\dot{\ast} \triangleq \frac{{\rm d} }{{\rm d} t}.$$</h3>
See the following link for [further description](resources/DA_intro.pdf#page=22). 

As a test case for DA, the state vector is <span style='font-size:1.25em'>$\mathbf{x} = (x,y,z)$</span>, and the parameters are typically set to

In [ ]:
SIGMA = 10.0
BETA  = 8/3
RHO   = 28.0

The equations relate the properties of a two-dimensional fluid layer uniformly warmed from below and cooled from above. In particular, the equations describe the rate of change of three quantities with respect to time: x is proportional to the rate of convection, y to the horizontal temperature variation, and z to the vertical temperature variation.  

The dynamics can be written as follows

In [ ]:
def dxdt(xyz, t0, sigma=SIGMA, beta=BETA, rho=RHO):
    """Compute the time-derivative of the Lorenz-63 system."""
    x, y, z = xyz
    return array([
        sigma * (y - x),
        x * (rho - z) - y,
        x * y - beta * z
    ])

#### Numerical computation of trajectories

Below is code to numerically integrate the differential equations and plot the solutions. This function has arguments that control the parameters of the differential equation <span style='font-size:1.25em'>$(\sigma,\beta,\rho)$</span>.  In the following we will study how small perturbations of a "control" trajectory change over time.

Additional parameters in the code inlcude:
<ul>
    <li> "N", defining the number of perturbations;</li>
    <li> "eps", defining the size of perturbations;</li>
    <li> "T", defining the length of the forward evolution.  **Note**: we will only plot the trajectory along times $[T-10, T]$ for $T>10$</li>
</ul>

**Exc 4.6**: Use the code below to investigate sensititivy to initial conditions.  Answer the following questions:
<ol>
    <li> For small pertubations, eps=0.01, how long does it take to see the nearby initial conditions lose track of the control trajectory?
    <li> Does it appear that there are parameter configurations where there are **stable** fixed points? </li>
    <li> Does it appear that there are parameter configurations where there are **stable** periodic behaviors?  **Hint**: what pattern emerges with $\rho = 350$?  What about $\rho=100.5$?</li>
    <li>When all trajectories are drawn to a limiting behavior, we call this behavior **globally stable**.  Do the periodic solutions appear to be globally or locally stable?  Try multiple values of epsilon, and consider what happens when there is more than one periodic behavior.</li>
</ol>

In [ ]:
def animate_lorenz(sigma=SIGMA, beta=BETA, rho=RHO, N=2, eps=0.01, T=0.1):    
    
    # Initial conditions: perturbations around some "proto" state
    seed(1)
    x0_proto = array([-6.1, 1.2, 32.5])
    x0 = x0_proto + eps*randn((N, 3))

    # Compute trajectories
    tt = linspace(0, T, int(T/.01)+1)               # Time instances for trajectory
    d2 = lambda x,t: dxdt(x,t, sigma,beta,rho)      # Define dxdt(x,t) with fixed params.
    xx = array([odeint(d2, x0i, tt) for x0i in x0]) # Integrate
    
    
    # PLOTTING
    ax = plt.figure(figsize=(16,8)).add_subplot(111, projection='3d')
    
    colors = plt.cm.jet(linspace(0,1,N))
    for i in range(N):
        # plot each ensemble member, but only the last 1000 steps of the trajectory
        ax.plot(*(xx[i,-1000:,:].T),'-'  ,c=colors[i])
        ax.scatter3D(*xx[i,-1,:],s=40,c=colors[i])

    ax.axis('off')
    plt.show()

    
w = interactive(animate_lorenz, sigma=(0.,50), rho=(0.,350, .5), beta=(0.,5),
                N=(1,10), eps=(0.01,10.01, .1),T=(0.1,100))

w

In the standard configuration of the Lorenz-63 model we see that small perturbations quickly diverge from control trajectories.  But even with perfect knowledge of the state of the atmosphere, our numerical approximations of its evolution would quickly degrade the forecast skill to zero.  As a proof of concept, suppose the "<b>true</b>" atmosphere is equal to the Lorenz-63 system, evolved via the simple <b>[forward Euler method](https://en.wikipedia.org/wiki/Euler_method)</b>, with a time step of 

In [ ]:
h_true = 0.0001

Suppose we are given the <b>exact</b> state of the atmosphere, defined as

In [ ]:
xyz_exact = array([-6.1, 1.2, 32.5])

But we must use the <b>[Order 4.0 Runge-Kutta](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods#The_Runge%E2%80%93Kutta_method)</b> scheme,

In [ ]:
def l63_rk4_step(xyz, h):
    """ calculate the evolution of Lorenz-63 one step forward via RK-4"""
    
    k_xyz_1 = dxdt(xyz, h, sigma=SIGMA, beta=BETA, rho=RHO)
    k_xyz_2 = dxdt(xyz + k_xyz_1 * (h / 2.0), h, sigma=SIGMA, beta=BETA, rho=RHO)
    k_xyz_3 = dxdt(xyz + k_xyz_2 * (h / 2.0), h, sigma=SIGMA, beta=BETA, rho=RHO)
    k_xyz_4 = dxdt(xyz + k_xyz_3 * h, h, sigma=SIGMA, beta=BETA, rho=RHO)

    xyz_step = xyz + (h / 6.0) * (k_xyz_1 + 2 * k_xyz_2 + 2 * k_xyz_3 + k_xyz_4)

    return xyz_step

with a time step of

In [ ]:
h_approximate = 0.1

to derive an <b>approximate forecast</b>, due to computational constraints.  Note that the discretization error of <b>both</b> the forward Euler (with time step h_true=0.0001) and the Order 4.0 Runge-Kutta (with time step h_approximate=0.1) is on the order $\mathcal{O}\left(10^{-4}\right)$.

**Exc 4.8**: Use the function "<b>dxdt</b>" defined above to code the forward Euler method.  Fill in the missing line in the code below. 

In [ ]:
def l63_forward_euler_step(xyz, h):
    """x_step is the one-step-forward state, derived from the initial condition xyz"""
    
    ### Fill in missing line here ###
    
    return xyz_step

In [ ]:
## Example solution

# show_answer('forward_euler')

**Exc 4.10**: Verify that your solution to Exc 4.6 works, using the GUI slider below.  The following code will generate the two paths from the **same** initial condition:
<ol>
    <li>the "true" atmosphere, generated by the forward Euler scheme and time step of 0.0001;</li>
    <li>the approximate atmospher, generated by the Runge-Kutta scheme with a time step of 0.1;</li>
</ol>
where the discretization error of each scheme is on the order of $\mathcal{O}\left(10^{-4}\right)$.


In [ ]:
def animate_approximation_divergence(T=0.1):    
    
    ## Compute trajectories
    xyz_approx_step = copy.copy(xyz_exact)
    xyz_true_step = copy.copy(xyz_exact)
    
    # define the number of integration steps
    true_steps = int(T / h_true)
    approx_steps = int(T / h_approximate)
    
    # define storage for the approximate trajectory
    xyz_approx = zeros([approx_steps + 1, 3])
    
    # define storage for the true trajectory, but where we only store the same time steps as the approximate one
    xyz_true = zeros([approx_steps + 1, 3])
    
    for i in range(approx_steps + 1):
        # store the value for each the true and approximate trajectory
        xyz_true[i, :] = xyz_true_step
        xyz_approx[i, :] = xyz_approx_step
            
        # forward propagate the approximate trajectory only at increments of 0.1
        xyz_approx_step = l63_rk4_step(xyz_approx_step, h_approximate)
        
        for j in range(1000):
            # forward propagate the true trajectory at every step of 0.0001
            xyz_true_step = l63_forward_euler_step(xyz_true_step, h_true)
        
            
            
    # PLOTTING
    ax = plt.figure(figsize=(10,5)).add_subplot(111, projection='3d')
    xx = np.dstack([xyz_true, xyz_approx])
    
    colors = plt.cm.jet(linspace(0,1,2))
    for i in range(2):
        # plot each of the trajectories, integrated with separate rules
        ax.plot(*(xx[-4:,:,i].T),'-'  ,c=colors[i])
        ax.scatter3D(*xx[-1,:,i], s=40, c=colors[i])

    ax.set_xlim((-15, 15))
    ax.set_ylim((-25, 25))
    ax.set_zlim((15, 40))
    ax.view_init(30, 120)
    ax.axis('off')
    plt.show()
    

w = interactive(animate_approximation_divergence, T=(0.4,4))
w

** Exc 4.12**: What do you notice about this plot?  What does this say about small simulation errors in chaotic systems?

## Dynamics of perturbations

In the following, we will neglect the important issues of model deficiencies and the inherent stochasticity of certain dynamics.  Instead, we will focus on the simplified scenario where we assume:
<ul>
    <li> we can perfectly model and compute the purely deterministic dynamics; and</li>
    <li> prediction error originates soley from the uncertainty in initial conditions.</li>
</ul>

Understanding that perturbations rapidly diverge even in a chaotic system as described above, this led to the transition from single-trajectory forecasts to ensemble-based forecasts.   Ensembles are used to "average out" our initialization errors, and to understand the variability and uncertainty in forecasts.

The advantages of ensemble based forecasts over single trajectory forecasts historically led to a search for perturbations that are most representative of the error growth in operational forecasts.  Prediction centers have sought to initialize ensemble-based forecasts in a way to best capture the variability induced by the dynamical chaos.  Two major techniques emerged,
<ol>
   <li> "<b>[bred vectors](https://journals.ametsoc.org/doi/abs/10.1175/1520-0477%281993%29074%3C2317%3AEFANTG%3E2.0.CO%3B2)</b>", and </li>
   <li> "<b>[forcing singular vectors](https://onlinelibrary.wiley.com/doi/abs/10.1034/j.1600-0870.1993.t01-4-00005.x)</b>". </li>
</ol>
   

These lead to different formulations of the classical "<b>[Lyapunov vectors](http://www.lmd.ens.fr/legras/publis/liapunov.pdf)</b>".  We do not stress here what a "Lyapunov vector" is, rather we will discover their nature experimentally in the following work.  This will lead to a formal definition of <em>one type</em> of Lyapunov vectors by the end of the exercises.

### "Breeding" growing modes

Suppose we have a smooth, nonlinear dynamical system,
<h3>
$$
\begin{align}
\dot{\mathbf{x}} = f(\mathbf{x}) & & \mathbf{x} \in \mathbb{R}^n,
\end{align}
$$
</h3>
and a precise estimate of an initial condition <span style='font-size:1.25em'>$\mathbf{x}^c_0$</span>, from which we want to make a forecast.  Suppose, also, that there are future observations that we will assimilate.

It was suggested by Toth and Kalnay to use the evolution of the initial estimate <span style='font-size:1.25em'>$\mathbf{x}^c_0$</span> as a control trajectory, while introducing small perturbations to generate an ensemble,
<h3>
$$\begin{align}
\mathbf{x}^i_0 = \mathbf{x}^c_0 + \boldsymbol{\delta}^i_0 
\end{align}$$
</h3>
where <span style='font-size:1.25em'>$\left \rvert \boldsymbol{\delta}^i \right \rvert  = \epsilon \ll 1$</span>.

The ensemble is evolved in parallel to the control trajectory.  Between times $t_{k-1}$ and $t_k$, this takes the form
<h3>
$$\begin{align}
\widehat{\mathbf{x}}^c_k &= \mathbf{x}_{k-1}^c + \int_{t_{k-1}}^{t_k} f(x) {\rm d}t \\
\widehat{\mathbf{x}}^i_k &= \mathbf{x}_{k-1}^i + \int_{t_{k-1}}^{t_k} f(x) {\rm d}t.
\end{align}
$$
</h3>

At the point of analyzing new observations we form a new estimate for the control trajectory, taking <span style='font-size:1.25em'>$\widehat{\mathbf{x}}_k^c$</span> to <span style='font-size:1.25em'>$\mathbf{x}_k^c$</span>. the perturbations are rescaled back to their original small size while maintaining their <em>directions</em>.  That is to say,
<h3>
$$
\begin{align}
\widehat{\boldsymbol{\delta}}_k^i \triangleq \mathbf{x}_k^c - \widehat{\mathbf{x}}_k^i, & &
\boldsymbol{\delta}_k^i \triangleq \frac{\epsilon}{\left\rvert \widehat{\boldsymbol{\delta}}^i_k\right\rvert} \widehat{\boldsymbol{\delta}}^i_k.
\end{align}
$$
</h3>

<a id='breeding'>"Breeding growing modes"</a> is designed to simulate how the modes of fast growing error are maintained and propagated through the successive use of short range forecasts.  The resulting perturbations are thus meant to represent a perturbation field of the "errors of the day", i.e., uncertainties in the initial condition at the present time that result from the repeated cycle of forecasts and analyses.

**Exc 4.14**: Run the code below and use the sliders to examine behavior of successive "breeding" of growing modes. The parameter **B** stands for the number of breeding cycles.  The parameter **eps** stands for the re-scaling parameter <span style='font-size:1.25em'>$\epsilon$</span> defined above.  The parameter **N** is the number of perturbations.  

The the plots on the left hand side show the evolution of the control trajectory and the perturbed trajectories along each breeding cycle.  The right hand side plots $\pm 1$ times the normalized perturbations,
<h3>
$$
 \frac{ \pm\widehat{\boldsymbol{\delta}}_k^i}{\left\rvert \widehat{\boldsymbol{\delta}}^i_k\right\rvert},
$$
</h3>
giving the **directions** of the perturbations, plotted as lines through the unit sphere. 

**Answer the following questions**:
<ol>
    <li> For small values of <span style='font-size:1.25em'>$\epsilon$</span>, what is significant about the long term behavior of the directions of the perturbations? </li>
    <li> Does this behavior change with large <b>N</b>, i.e., more directions for the pertubations? </li>
    <li> How does this behavior change when <span style='font-size:1.25em'>$\epsilon$</span> is increased?  Do the directions of the perturbations depend on **N** for large <span style='font-size:1.25em'>$\epsilon$</span>?
</ol>

In [ ]:
def animate_bred_vectors(B=0, eps=0.01, N=10):    
    
    # Initial conditions: perturbations around some "proto" state
    sigma=SIGMA 
    beta=BETA 
    rho=RHO
    T=0.05
    
    seed(1)
    x_0 = array([-6.1, 1.2, 32.5])               # define the control
    
    # define the perturbations, randomly generated but of fixed norm epsilon
    perts = randn([N,3])
    perts = array([eps * perts[i] / sqrt(perts[i] @ perts[i]) for i in range(N)])
    delta_x = x_0 + perts
                  
    tt = linspace(0, T, 10)           # Time instances for trajectory
    d2 = lambda x,t: dxdt(x,t, sigma,beta,rho)  # Define dxdt(x,t) with fixed params.    
    
    # for each breeding cycle
    for kk in range(B):
        # Compute trajectories
        x_traj = array([odeint(d2, x_0, tt)])        # integrate the control trajectory
        x_0 = np.squeeze(x_traj[:, -1, :])
        
        delta_x_traj = array([odeint(d2, delta_xi, tt) for delta_xi in delta_x]) # Integrate the perturbations
        perts = delta_x_traj[:, -1, :] - x_0                                     # redefine the perts
        perts = array([eps * perts[i] / sqrt(perts[i] @ perts[i]) for i in range(N)])
        delta_x = x_0 + perts # redefine the initialization of the perturbed trajectories
    
    # PLOTTING
    fig = plt.figure(figsize=(16,8))
    ax1 = plt.subplot(121, projection='3d')
    ax2 = plt.subplot(122, projection='3d')
    
    if B==0:
        ax1.scatter3D(*x_0, s=40, c='k')

    else:
        ax1.plot(*x_traj[0,:,:].T, '-', c='k')
        ax1.scatter3D(*x_traj[0,-1,:].T, '-', s=40, c='k')
            
    colors = plt.cm.jet(linspace(0,1,N))
    for i in range(N):
        # for each breeding cycle
        if B==0:
            # if just the initial conditions, we plot these
            ax1.scatter3D(*delta_x[i,:],s=40,c=colors[i])
            
        else:
            # otherwise, plot the trajectories over a breeding cycle
            ax1.plot(*(delta_x_traj[i,:,:].T),'-'  ,c=colors[i])
            ax1.scatter3D(*delta_x_traj[i,-1,:],s=40,c=colors[i])
                          
        # we plot the normalized perturbations on the unit sphere
        tmp = perts[i,:]/sqrt(perts[i,:] @ perts[i, :])
        p_vect = np.concatenate([np.reshape([0,0,0],[1,3]), np.reshape(tmp,[1,3])], axis=0)
        
        # delta * +1
        ax2.plot(p_vect[:,0], p_vect[:,1], p_vect[:,2],'-'  ,c=colors[i])
        ax2.scatter3D(*tmp[:],s=40,c=colors[i], marker='o')

        # delta * -1
        ax2.plot(-1*p_vect[:,0], -1*p_vect[:,1], -1*p_vect[:,2],'-'  ,c=colors[i])
        ax2.scatter3D(*tmp[:]*(-1),s=40,c=colors[i], marker='o')

    ax1.axis('off')
    ax2.set_xlim((-.9, .9))
    ax2.set_ylim((-.9, .9))
    ax2.set_zlim((-.9, .9))
    plt.show()
    
w = interactive(animate_bred_vectors,B=(0,175,25), eps=(0.01,1.61, .2), N=(1, 161, 20))
w

**Exc 4.16**: If the "breeding" of perturbations is meant to represent the unstable growth of initial perturbations, what can we learn from their growth rates?  In the following code, fill in the missing lines to define a function that will compute the log-growth rate of the perturbation <span style='font-size:1.25em'>$\boldsymbol{\delta^i}_k$</span>, i.e., the log growth relative to the length of time in the breeding interval.  This function should return
<h3>
$$\begin{align}
\frac{1}{T}\log \left( \frac{\left\rvert \widehat{\boldsymbol{\delta}}^i_k\right \rvert}{\left\rvert \boldsymbol{\delta}_{k-1}^i \right\rvert}\right) \equiv \frac{1}{T}\log \left( \frac{\left\rvert \widehat{\boldsymbol{\delta}}^i_k\right \rvert}{\epsilon}\right),
\end{align}$$
</h3>
for a single perturbation.

In [ ]:
def log_growth(x_control_k, x_pert_k, T, eps):
    """function returns array of the log growth values for a single perturbation"""
    
    
    ### Fill in missing line(s) here ###
    
    return log_growth_rate

In [ ]:
## Example solution

# show_answer('log_growth')

**Exc 4.18**: Test your answer to **Exc 4.16**.  Using the code and slider below, investigate the distributions of the log-growth rates of the bred vectors as a function of the number of breeding cycles.  Answer the following questions:
<ol>
    <li> What is the long term behaviour of this distribution?</li>
    <li> The leading Lyapunov exponent of the Lorenz-63 system is $\approx 0.9050$, what do you notice about the mean of the log-growth rates over a long number of breeding cycles?</li>
    <li> Consider the behavior of small perturbations from **Exc 4.14**. Can you conjecture what the perturbations are converging to?</li>
    <li> What does this suggest about the "representative perturbations" for the error growth in chaotic systems?</li>
</ol>

In [ ]:
def animate_bred_growth_rates(B=1000):    
    
    # Initial conditions: perturbations around some "proto" state
    sigma=SIGMA 
    beta=BETA 
    rho=RHO
    eps=0.01
    N=1
    T=0.01
    
    seed(1)
    x_0 = array([-6.1, 1.2, 32.5])               # define the control
    
    # define the perturbation, randomly generated but of fixed norm epsilon
    perts = randn(3)
    perts = eps * perts / sqrt(perts @ perts)
    delta_x = x_0 + perts
                  
    tt = linspace(0, T, 20)           # Time instances for trajectory
    d2 = lambda x,t: dxdt(x,t, sigma,beta,rho)  # Define dxdt(x,t) with fixed params.    
    grwt = np.zeros(B)
    
    
    # for each breeding cycle
    for kk in range(B):
        # Compute trajectories
        x_traj = array([odeint(d2, x_0, tt)])        # integrate the control trajectory
        x_0 = np.squeeze(x_traj[:, -1, :])
        
        delta_x_traj = array([odeint(d2, delta_x, tt)]) # Integrate the perturbation
        
        # compute the log growth from the code defined earlier
        grwt[kk] = log_growth(x_0, delta_x_traj[0, -1, :], T, eps)
        
        # redefine perts
        perts = delta_x_traj[0, -1, :] - x_0
        perts = eps * perts / sqrt(perts @ perts.T) 
        delta_x = x_0 + perts
    
    # PLOTTING
    fig = plt.figure(figsize=(16,8))
    ax = plt.subplot(111)
    ax.hist(grwt, bins=linspace(-20,20,4001), normed=True)
    ax.set_xlim((-12, 12))
    ax.set_ylim((0, 0.8))
    ax.text(4, 0.6, 'Mean log-growth rate=' + str(np.round(mean(grwt),decimals=4)).zfill(4), size=20)
    
    plt.show()
    
w = interactive(animate_bred_growth_rates,B=(1000,30000, 5800))
w

### Lyapunov exponents and eigenvalues

A **Lypunov exponent** can be understood loosely as a kind of generalized eigenvalue for time-depenent linear transformations, or for the linearization of a nonlinear evolution.

What do eigenvalues tell us about a matrix and why might the above results seem intuitive? 

Consider the <a id='perturbation_equation'>equation for the <em>evolution</em> of the pertubation <span style='font-size:1.25em'>$\boldsymbol{\delta}^i_k$</span></a>.  We can write,
<h3>
$$\begin{align}
& \boldsymbol{\delta}_k^i = \mathbf{x}_k^c - \mathbf{x}_k^i \\
\Rightarrow & \dot{\boldsymbol{\delta}}_k^i = f(\mathbf{x}_k^c) - f(\mathbf{x}_k^i).
\end{align}$$
</h3>
But for small perturbations, we can reasonably make an approximation with a Taylor expansion,
<h3>
$$\begin{align}
 f(\mathbf{x}_k^c) - f(\mathbf{x}_k^i) \approx \nabla f\rvert_{\mathbf{x}c}  \boldsymbol{\delta}^i_k, & &   
\end{align}$$
</h3> 
where the term, 
<h2>
$$f\rvert_{\mathbf{x}c}$$
</h2>
is the gradient with respect to the state variables, i.e., the **[Jacobian matrix](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant)**, evaluated at the control trajectory.

This means that for small perturbations, the evolution is well approximated by the linear Jacobian equations, and we can think of these linear equations having some kind of generalized eigenvalues, describing the invariant (exponential) growth and decay rates for the system. 

#### The power method

The method of breeding errors above is conceptually very similar to the classical [power method](https://en.wikipedia.org/wiki/Power_iteration) for finding the leading eigenvalue of a diagonalizable matrix:

* Suppose <span style='font-size:1.25em'>$\mathbf{M}\in\mathbb{R}^{n\times n}$</span> is a diagonalizable matrix, with eigenvalues,
<h3>
$$
\rvert \mu_1 \rvert > \rvert\mu_2\rvert \geq \cdots \geq \rvert\mu_n\rvert,
$$
</h3>
i.e., <span style='font-size:1.25em'>$\mathbf{M}$</span> has a single eigenvalue of magnitude greather than all its others.   

* Let <span style='font-size:1.25em'>$\mathbf{v}_0 \in \mathbb{R}^n$</span> be a randomly selected vector, with respect to the Gaussian distribution on <span style='font-size:1.25em'>$\mathbb{R}^n$</span>. 

* We define the algorithm,
<h3>
$$\begin{align}
\mathbf{v}_{k+1} \triangleq \frac{\mathbf{M} \mathbf{v}_k}{ \left\rvert \mathbf{M} \mathbf{v}_k\right\rvert} & &
\widehat{\mu}_{k+1} \triangleq \mathbf{v}_{k+1}^{\rm T} \mathbf{M} \mathbf{v}_{k+1} 
\end{align}$$
</h3>
as the power method.  

It is easy to verify that with probability one, the sequence <span style='font-size:1.25em'>$\widehat{\mu}_k$</span> converges to the dominant eigenvalue, <span style='font-size:1.25em'>$\mu_1$</span>, and <span style='font-size:1.25em'>$\mathbf{v}_k$</span> converges to an eigenvector for the dominant eigenvalue. 

**Exc 4.20**: Fill in the code below to write an algorithm for the power method.

In [ ]:
def power_method(M, v, number_iterations):
    """takes a diagonalizable matrix M and returns approximations for the leading eigenvector/eigenvalue"""
    
    for i in range(number_iterations):
        ### fill in missing lines here
       
    return v, mu

In [ ]:
# Example solution

# show_answer('power_method')

**Exc 4.22**: Test your solution to **Exc 4.20**.  Use the code and slider below to study the rate of convergence.  In this case, the matrix will have eigenvalues 
<h3>$$\begin{align}
\left\{r^i : \hspace{2mm} i =0, 1, 2, \hspace{2mm} \text{and} \hspace{2mm} r\in(1,2]\right\}
\end{align}$$</h3>
The parameter <span style='font-size:1.25em'>$k$</span> defines how many iterations of the power method are computed.  How does the value <span style='font-size:1.25em'>$r$</span> affect the number of iterations necessary to reach convergence?

In [ ]:
def animate_power_convergence_rate(k=1, r=1.5):    
    
    # We define a well conditioned matrix M, depending on the ratio of the eigenvalues
    M = array([r ** i for i in range(3)])
    M = np.diag(M)
    e_3 = array([0, 0, 1])

    # define a random initial condition
    np.random.seed(0)
    v = randn(3)
    v = v / sqrt(v.T @ v)
    
    # and storage for the series of approximations
    v_hist = zeros(k+1)
    v_hist[0] = e_3.T @ v 
    
    mu_hist = zeros(k+1)
    mu_hist[0] = v.T @ M @ v
    
    # for the number of iterations k, return the power method approximation
    for it in range(1,k+1):
        np.random.seed(0)
        v, mu = power_method(M, v, it)
        v_hist[it] = np.arccos(e_3.T @ v)
        mu_hist[it] = mu
     
    # PLOTTING
    fig = plt.figure(figsize=(16,8))
    ax1 = plt.subplot(121)
    ax2 = plt.subplot(122)
    ax1.plot(range(0,k+1), v_hist)
    ax2.plot(range(0,k+1), mu_hist)
    ax1.set_ybound([0,1.05])
    ax2.set_ybound([.9,4])
    
    t_scl = np.floor_divide(k+1, 10)
    
    ax1.set_xticks(range(0, k+1, t_scl + 1))
    ax2.set_xticks(range(0, k+1, t_scl + 1))
    
    ax1.text(0, 1.07, r'Angle between $\mathbf{v}_k$ and eigenvector', size=20)
    ax2.text(0, 4.05, r'Value of $\mu_k$', size=20)
    ax1.tick_params(
        labelsize=20)
    ax2.tick_params(
        labelsize=20)
    
    plt.show()

w = interactive(animate_power_convergence_rate, k=(1,15), r=(1.05,2, .05))
w

**Exc 4.24.a **: Suppose the power method is performed on a generic diagonalizable matrix <span style='font-size:1.25em'>$\mathbf{M}\in\mathbb{R}^{n\times n}$</span>, with eigenvalues
<h3>$$\begin{align}
\rvert \mu_1 \rvert > \rvert\mu_2 \rvert\geq \cdots \geq \rvert\mu_n \rvert,
\end{align}$$</h3>
with a randomly selected initial vector <span style='font-size:1.25em'>$\mathbf{v}_0$</span>, with respect to the Gaussian distribution on <span style='font-size:1.25em'>$\mathbb{R}^n$</span>.

Can you conjecture what is the order of convegence for the sequences <span style='font-size:1.25em'>$\mathbf{v}_k$</span> and <span style='font-size:1.25em'>$\widehat{\mu}_k$</span>? 

**Hint**: the rate depends on the eigenvalues.

**Exc 4.42.b***: Prove the rate of convergence.

In [ ]:
# Answer

# show_answer('power_method_convergence_rate')

** Exc 4.28* **: We have brushed over why the algorithm described above converges with *probability one*, can you prove why this is the case?

In [ ]:
# Answer

# show_answer('probability_one')

** Exc 4.30.a **: Let <span style='font-size:1.25em'>$\widehat{\mu}_k$</span> be defined as in **Exc 4.24**.  Suppose we define a sequence of values,
<h3>$$\begin{align}
\widehat{\lambda}_T = \frac{1}{T} \sum_{k=1}^T\log\left(\rvert \widehat{\mu}_k\right \rvert).
\end{align}$$</h3>
Answer the following:
<ol>
  <li> Can you conjecture what <span style='font-size:1.25em'>$\widehat{\lambda}_T$</span> converges to as <span style='font-size:1.25em'>$T \rightarrow \infty$</span>?  
  
  **Hint**: Use the fact that <span style='font-size:1.25em'>$\widehat{\mu}_k \rightarrow \mu_1$</span> as <span style='font-size:1.25em'>$k \rightarrow \infty$</span></li>
  
  <li> Suppose we define the Lyapunov exponents as the log-average growth rates of the matrix <span style='font-size:1.25em'>$\mathbf{M}$</span>.  What can you guess about the relationship between the eigenvalues and the Lyapunov exponents of the matrix <span style='font-size:1.25em'>$\mathbf{M}$</span>?</li>
<ol>

** Exc 4.30.b* **: Prove that the limit
<h3>$$\begin{align}
\lim_{T \rightarrow \infty} \widehat{\lambda}_T
\end{align}$$</h3>
exists, and what quantity it converges to.

In [ ]:
# Answers

# show_answer('lyapunov_exp_power_method')

#### The QR algorithm

The power method is an intuitive method for finding the dominant eigenvalue for a special class of matrices.  However, we generally want to find directions that may also be growing, though more slowly than the dominant direction.  

Intuitively, if we are tracking a control trajectory with data assimilation and we corrected the forecast errors only in the direction of dominant error growth, we may still lose track of the control trajectory, only it would be more slowly than the dominant rate of growth.

There is a simple generalization of the power method for finding higher dimensional subspaces.  We may consider *separating* perturbations into directions that grow at different rates.  One easy way to perform this is to construct a *moving frame* in the span of the perturbations.  If there is only one perturbation, then the power method constructs precisely a 1-dimensional moving frame, with a vector that is always of norm 1. 

If there are two perturbations we can construct a moving frame in the span of the perturbations with a [Gram-Schmidt](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process) step.  A visualization of the Gram-Schmidt process for three vectors is picuted in the visualization below.

<div style='width:900px'>
<img src="./resources/Gram-Schmidt_orthonormalization_process.gif">
</div>

**By Lucas V. Barbosa [Public domain], <a href="https://commons.wikimedia.org/wiki/File:Gram-Schmidt_orthonormalization_process.gif">from Wikimedia Commons</a>**

In our case, suppose we have two initial, orthogonal vectors
<h3>$$
\mathbf{x}_0^1, \mathbf{x}_0^2
$$</h3>
which we will propagate forward.  We define for each $j=1,2$,
<h3>$$
\widehat{\mathbf{x}}^j_1 \triangleq \mathbf{M} \mathbf{x}^j_0.
$$</h3>
The first vector will follow the usual power method, i.e.,
<h3>$$
\mathbf{x}^1_1 \triangleq \frac{\widehat{\mathbf{x}}_1^1}{\left\rvert \widehat{\mathbf{x}}_1^1\right\rvert},
$$</h3>

However, we want to separate the second vector <span style='font-size:1.25em'>$\widehat{\mathbf{x}}_1^2$</span> so the new perturbations don't align.  We thus remove the components in the direction of <span style='font-size:1.25em'>$\mathbf{x}_1^1$</span>, before we normalize <span style='font-size:1.25em'>$\widehat{\mathbf{x}}_1^2$</span>.  
<h3>$$\begin{align}
\mathbf{y}^2_1 &\triangleq \widehat{\mathbf{x}}_1^2- \langle \mathbf{x}_1^1,  \widehat{\mathbf{x}}^2_1\rangle \mathbf{x}_1^1 \\
\mathbf{x}^2_1 & \triangleq \frac{\mathbf{y}_1^2}{\left\rvert \mathbf{y}_1^2 \right\rvert}
\end{align}$$</h3>

It is easy to see by definition that <span style='font-size:1.25em'>$\mathbf{x}_1^1, \mathbf{x}_1^2$</span> are orthogonal, but we can also show an important dynamical property with this transformation.  Define the following coefficients,
<h3>$$
\begin{align}
U^{11}_1 &=\left\rvert \widehat{\mathbf{x}}_1^1\right\rvert \\
U^{22}_1 &=\left\rvert \mathbf{y}_1^2 \right\rvert \\
U^{12}_1 &= \langle \mathbf{x}^1_1, \mathbf{x}_1^2\rangle 
\end{align}
$$<h3>

**Exc 4.32**: Can you write the recursion for the vectors <span style='font-size:1.25em'>$\mathbf{x}_0^1, \mathbf{x}_0^2$</span> transformed into <span style='font-size:1.25em'>$\mathbf{x}_1^1,\mathbf{x}_1^2$</span> with the coefficients <span style='font-size:1.25em'>$U^{ij}_1$</span> defined above in matrix form?  Can you write the recursion for an arbitrary number of steps $k\in\{1,2,\cdots\}$?

In [ ]:
# Answer

# show_answer('gram-schmidt')

The above procedure defines the *naive* QR algorithm --- one should note that there are more computationally efficient versions of this algorithm utilized in standard linear algebra software libraries.  However, this simple intuition forms the basis for many powerful theoretical results.  

The QR algorithm (in its refined version) is the standard method for computing the <b>[Schur decomposition](https://en.wikipedia.org/wiki/Schur_decomposition)</b> for a matrix, which is used for many purposes as it is a numerically stable alternative to the <b>[Jordan Cannonical Form](https://en.wikipedia.org/wiki/Jordan_normal_form)</b>, pictued below:

<div style='width:900px'>
<img src="./resources/Jordan_blocks.svg">
</div>

**By Jakob.scholbach [<a href="https://creativecommons.org/licenses/by-sa/3.0">CC BY-SA 3.0</a> or <a href="http://www.gnu.org/copyleft/fdl.html">GFDL</a>], <a href="https://commons.wikimedia.org/wiki/File:Jordan_blocks.svg">from Wikimedia Commons</a>**

The Jordan Canonical form is highly appealing as it is the diagonal or "almost-diagonal" form of a matrix.  However, this is highly unstable to compute in most applications.

The Schur decomposition relaxes this further, from "almost-diagonal" to upper triangular, another useful form for a matrix.  In particular, the Schur decomposition is one approach to find **all eigenvalues** for a matrix, separated into a **chain of descending growth and decay rates**.

** Exc 4.34**: Suppose a matrix <span style='font-size:1.25em'>$\mathbf{M}$</span> has a Schur decomposition, given as,
<h3> $$ \begin{align}
\mathbf{M} = \mathbf{Q} \mathbf{U} \mathbf{Q}^{\rm T},
\end{align}$$ </h3>
where <span style='font-size:1.25em'>$\mathbf{U}$</span> is strictly upper triangular, and <span style='font-size:1.25em'>$\mathbf{Q}$</span> is orthogonal such that <span style='font-size:1.25em'>$\mathbf{Q}^{\rm T} = \mathbf{Q}^{-1}$</span>.  Can you prove that the eigenvalues of <span style='font-size:1.25em'>$\mathbf{M}$</span> are the diagonal elements of <span style='font-size:1.25em'>$\mathbf{U}$?</span> 

If <span style='font-size:1.25em'>$\mathbf{Q}^j$</span> is the $j$-th column of <span style='font-size:1.25em'>$\mathbf{Q}^j$</span>, what does the product
<h3>$$\begin{align}
\left(\mathbf{Q}^j\right)^{\rm T} \mathbf{M} \mathbf{Q}^j
\end{align}$$</h3>
equal in terms of the earlier quantities? ** Hint**: how does this relate to the power method?

In [ ]:
# Answer

# show_answer('schur_decomposition')

** Exc 4.36**: Can you conjecture what the Schur decomposition will take in the case that the matrix <span style='font-size:1.25em'>$\mathbf{M}$</span> has complex eigenvalues?

In [ ]:
# Answer

# show_answer('real_schur')

### A construction for Lyapunov exponents and "Lyapunov vectors"

We return now to the discussion of perturbations in nonlinear models.  Recall the [equations](#perturbation_equation) for the evolution of perturbations.  We may define *linear* dynamics, generated by the Jacobian equation,
<h2>$$\begin{align}
\frac{{\rm d} \mathbf{x}}{{\rm d} t} = \nabla f_{\rvert_{\mathbf{x}c}},
\end{align}$$</h2>
computed along some control trajectory <span style='font-size:1.25em'> $\mathbf{x}^c(t)$</span>.  This system of equations is known as the <b>[tangent-linear model](http://glossary.ametsoc.org/wiki/Tangent_linear_model)</b>.  The tangent space can be understood as the **space of perturbations** at a point (or along a trajectory).  It can be used to define a vector field on a "space" describing the time evolution of trajectories. 

<div style='width:900px'>
<img src="./resources/Tangentialvektor.svg">
</div>

**By derivative work: McSush (talk)Tangentialvektor.png: TNThe original uploader was TN at German Wikipedia (Tangentialvektor.png) [Public domain], <a href="https://commons.wikimedia.org/wiki/File:Tangentialvektor.svg">via Wikimedia Commons</a>**

As we have seen with the [breeding of errors](#breeding), we can compute the log-average growth rate of the dominant growing mode of the tangent-linear model to approximate the leading Lyapunov exponent.  In a linear system, with fixed matrix <span style='font-size:1.25em'> $\mathbf{M}$</span> this corresponds exactly to taking the log-average growth rate in the power method.  Recall the generalization of the power method to the QR algorithm.  We might hope to find **all of the log-average growth rates** in the tangent-linear model by computing the log averages of the diagonal elements in the QR factors <span style='font-size:1.25em'> $\mathbf{U}_k$</span>. 

In this case, the Gram-Schmidt factors form a basis for the tangent linear model where:
<ol>
    <li> the leading vector aligns with the dominant growth mode;</li>
    <li> the subsequent vectors separate out the lower order growth rates.</li>
</ol>

Suppose the tangent-linear model is computed discretely in time, where <span style=font-size:1.25em> $\textbf{M}_k$</span> takes the perturbations at time <span style=font-size:1.25em>$t_{k-1}$</span> to time <span style=font-size:1.25em>$t_k$</span>.  Suppose that in matrix form, <span style=font-size:1.25em>$\mathbf{M}_k$</span>, produces the following QR factorization,
<h3>$$\begin{align}
\mathbf{M}_k \mathbf{E}_{k-1} &= \mathbf{E}_k \mathbf{U}_k
\end{align}$$</h3>

**Exc 4.36**: Suggest a constructive definition for:
<ol>
    <li> the Lyapunov exponents of the nonlinear model</li>
    <li> the "Lyapunov vectors" </li>
</ol>
**Note:** We will not stress yet what kind of Lyapunov vector we are constructing.  It turns out that Lyapunov exponents are generally **globally defined**, but there are several types of Lyapunov vectors that are locally defined.

In [ ]:
# Answer 

# show_answer('lyapunov_vs_es')

The type of vector this construction leads to is the **"backward" Lyapunov vectors**.  They are denoted "backward" because they contain information from the in the past, leading to the current time, i.e., the "errors of the day" described by Toth and Kalnay.  

For completeness, we remark that the forcing singular vectors mentioned above can be shown to converge to the "forward" Lyapunov vectors.  These are denoted "forward" because they contain information about the future evolution of pertubations, pulled back to the current time.  A third type of Lyapunov vector is also often studied, which are called "covariant".  Similar to how eigenvectors always are mapped to a scaled copy of themselves, covariant vectors share an analogous property with respect to time-varying dynamics.  

A full discussion of the significance of forward, backward, and covariant vectors goes well beyond the scope of this tutorial.  For a comprehensive discussion of Lyapunov theory, aimed at practicioners, it is recommended to read the work of [Legras & Vautard](http://www.lmd.ens.fr/legras/publis/liapunov.pdf) and [Kutpsov & Parlitz](https://arxiv.org/abs/1105.5228).

## Ensemble based covariances

When we restrict the forecasting problem to the situation where we assume:
<ul>
    <li> we can perfectly model and compute the purely deterministic dynamics; and</li>
    <li> prediction error originates soley from the uncertainty in initial conditions,</li>
</ul> 
it is realistic to think of the data assimilation problem as tracking the evolution of perturbations along a control trajectory.  This control trajectory is the "true" state of the dynamical system we are studying, which we receive observations of, and try to predict the behavior of at future times.

In this case, we suppose we have a prior distribution for the state of the dynamical system.  Suppose we sample for an ensemble of "nearby" initial conditions.  If the ensemble remains sufficiently close to the control trajectory, and we can use the [approximation for the evolution of perturbations](#perturbation_equation) to accurately model the forecast errors, then the ensemble spread will be characterized by the backward Lyapunov vectors and their growth and decay rates.

Consider the conceptual image below.  Suppose the initial prior covariance is given by the unit disk, centered on the "true" state of a dynamical system.  If the evolution of uncertainty can be well approximated by the tangent linear model along the "truth", we will expect the covariance to stretch and deform into an ellipse according to the directions of **growth** and **decay**.
<div style='width:800px'>
<img src="./resources/LyapunovDiagram.svg">
</div>
**By Mrocklin (original creation) [<a href="https://creativecommons.org/licenses/by-sa/3.0">CC BY-SA 3.0</a> or <a href="http://www.gnu.org/copyleft/fdl.html">GFDL</a>], <a href="https://commons.wikimedia.org/wiki/File:LyapunovDiagram.svg">via Wikimedia Commons</a>**

In the following, we will implement a simple, "square-root" ensemble Kalman filter in the Lorenz-63 model.  The ensemble Kalman filter will be given **noisy observations** of a control trajectory.   Along the control trajectory, we will compute the **QR factorization** of the tangent-linear model.  

We will define the **projection coefficient** of the covariance matrix <span style='font-size:1.25em'> $\mathbf{P}_k$</span> into the $j$-th QR factor to be the quantity,
<h3>$$\begin{align}
\left(\mathbf{Q}_k^j\right)^{\rm T} \mathbf{P}_k \mathbf{Q}^j
\end{align}$$</h3>
We will compute the projection coefficients of the ensemble based covariance into each of the QR factors, as defined above.  

** Exc 4.38**: Can you conjecture how the projection coefficients will vary in the index $j$? <br>
**Hint**: consider **Exc 4.34**.

** Exc 4.40**: Test your conjecture from **Exc 4.40**.  Use the code below to investigate the relationship between the ensemble based covariance and its projeciton into each of the QR factors.  We plot the average projection coefficient for the EnKF covariance into each of the QR factors over the number of analyses.  Similarly, we plot the EnKF mean square error, to vefify that the ensemble mean lies within the variance of the error in these directions.

**Note**: we may consider the QR factorizations to give approximately the "true" Lyapunov vectors when the log-average growth rate approaches the Lyapunov exponents for the system.  The Lyapunov exponents are approximately given by,
<h3>$$\begin{align}
\lambda_1 &\approx 0.905 \\
\lambda_2 & = 0 \\
\lambda_3 & \approx -14.571.
\end{align}$$<h3/>

In [ ]:
sigma=SIGMA 
beta=BETA 
rho=RHO

def l63_jac(x):
    jac = np.array([
        [-sigma,  sigma,     0 ],
        [rho - x[2], -1,  -x[0]],
        [x[1],      x[0], -beta]
        ]
    )
    
    return jac

def l63_step_TLM(x, Y, h):
    
    h_mid = h/2

    # calculate the evolution of x to the midpoint
    x_mid = l63_rk4_step(x, h_mid)

    # calculate x to the next time step
    x_next = l63_rk4_step(x_mid, h_mid)

    k_y_1 = l63_jac(x).dot(Y)
    k_y_2 = l63_jac(x_mid).dot(Y + k_y_1 * (h / 2.0))
    k_y_3 = l63_jac(x_mid).dot(Y + k_y_2 * (h / 2.0))
    k_y_4 = l63_jac(x_next).dot(Y + k_y_3 * h)

    Y_next = Y + (h / 6.0) * (k_y_1 + 2 * k_y_2 + 2 * k_y_3 + k_y_4)

    return [x_next, Y_next]

def animate_enkf_covariance(nanl=0):    
    
    # Initial conditions: perturbations around some control state
    tanl=0.005
    h = 0.001
    tl_steps = int(tanl / h)
    N = 4
    obs_un = 0.25
    obs_dim = 3
    R = np.eye(obs_dim) * obs_un
    H = np.eye(3, M=obs_dim).T
    proj_traj = np.zeros([nanl, 3])
    err = np.zeros([nanl])
    
    seed(1)
    x_0 = array([-6.1, 1.2, 32.5])               # define the control
    
    # define the perturbations, randomly generated but of fixed norm epsilon
    A_f = randn([3, N])
    a_m = np.mean(A_f, axis=1)
    A_f = A_f.T - a_m
    del a_m
    A_f = (x_0 + A_f).T
                  
    lam = np.zeros(3)
    Q = np.eye(3)
        
    # for each analysis cycle
    for kk in range(nanl):
        for j in range(tl_steps):
            x_0, Q = l63_step_TLM(x_0, Q, h)
            for l in range(2):
                for j in range(N):
                    A_f[:, j] = l63_rk4_step(A_f[:, j], h / 2)
            
        # perform QR step and find the local Lyapunov exponents
        Q, U = np.linalg.qr(Q)
        lam += np.log(np.abs(np.diag(U))) / tanl
        
        # define an observation
        y_0 = H @ x_0 + np.random.multivariate_normal(np.zeros([obs_dim]), R)
        
        # forecast mean, and rmse
        x_f = np.mean(A_f, axis=1)
        
        A_f = (A_f.T - x_f).T
        
        # forecast covaraince
        P_f = (N-1) ** (-1) * A_f @ A_f.T 
        
        # form the Kalman gain    
        K = P_f @ H.T @ np.linalg.inv(H @ P_f @ H.T + R) 
        
        # analysis mean
        x_a = x_f + K @ (y_0 - H @ x_f)
        err[kk] = np.mean((x_a - x_0)**2)
        
        # analyze the ensemble
        T = np.eye(N) - (N-1)**(-1) * (H @ A_f).T @ np.linalg.inv(H @ P_f @ H.T + R) @ (H @ A_f)
        U, S, V_h = np.linalg.svd(T)
        T = U @ np.diag(np.sqrt(S)) @ U.T
        A_f = A_f @ T
        
        P_a = (N-1)**(-1) * A_f @ A_f.T
        # find the forecast projection coefficients
        for i in range(3):
            proj_traj[kk, i] = Q[:, i].T @ P_a @ Q[:, i]
            
        A_f = (x_a + A_f.T).T
        
        
    # PLOTTING
    avg_proj = np.zeros([nanl, 3])
    
    # we plot the average projection coefficient into each blv
    for i in range(1,nanl):
        avg_proj[i, :] = np.mean(proj_traj[:i, :], axis=0)
    
    lam = lam / nanl
    fig = plt.figure(figsize=(16,8))
    ax = plt.subplot(111)
    
    for i in range(3):
        ax.plot(range(nanl), avg_proj[:, i], label='BLV projection ' + str(i + 1) + ', log-avg growth rate ' + 
                str(np.round(lam[i],decimals=3)).zfill(3))
                
    ax.axhline(y=np.mean(err), color='k', label='Mean square error')
    plt.legend(fontsize=20)
    ax.set_xbound([1, nanl])
    ax.set_yscale('log')
    ax.tick_params(labelsize=20)
    plt.show()
    
w = interactive(animate_enkf_covariance,nanl=(10,20010,2000))
w

** Exc 4.42**: Answer the following questions.
<ol>
   <li>How do the projection coefficients relate to the log-average growth rates in each direction? </li> 
   <li>What is significant about the **effective rank** of the covariance?</li>
   <li>Can you conjecture what this means about the necessary number of ensemble members to prevent filter divergence?
   <br> **Hint**: the ensemble should capture the effective spread of the uncertainty.</li>
</ol>

### Next: [Ensemble [Monte-Carlo] approach](T5 - Ensemble [Monte-Carlo] approach.ipynb)